In [ ]:
# Time when the script starts
import time
start = time.time()
print('Script started at time =', start)

# Whole cell segmentation
We have modified this notebook from the original version. Refer to ``deepcell-tf`` documentation for more information: https://deepcell.readthedocs.io/.

### Initialize Mesmer model

The application will download pretrained weights for tissue segmentation as well as other packages that facilitate the cell segmentation process. For more information about application objects, please see the original [documentation](https://deepcell.readthedocs.io/en/master/API/deepcell.applications.html). Any warning messages that appear after running this is fine.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from deepcell.applications import Mesmer
from deepcell.datasets import multiplex_tissue
from deepcell.utils.plot_utils import create_rgb_image
from deepcell.utils.plot_utils import make_outline_overlay
from deepcell.utils.misc_utils import sorted_nicely
from matplotlib import pyplot as plt
from tifffile import TiffFile
import tifffile as tiff
import numpy as np

app = Mesmer()

### Set parameters

In the cell below, edit the following values according to the comments in the cell. This enables the script to: 1. Locate our nucleus.tif and surface.tif files, 2. Define the mpp used for cell segmentation, 3. Save the cellSegmentation.tif and cellOutline.tif files (into the same folder).

In [ ]:
# Set 'path' to the location of the folder that contains the nucleus.tif and surface.tif files
path = '/Users/Project/Analysis/'

# Set 'allFolders' to the name of the folder containing the .tif files
allFolders = os.listdir(path)
allFolders = ['images']

# If necessary, set 'cellNuc' to the name of the nucleus.tif file (exclude '.tif')
# If necessary, Set 'cellSur' to the name of the surface.tif file (exclude '.tif')
cellNuc = 'nucleus'
cellSur = 'surface'

# Our recommended range of mpp values to test: 0.2 to 2.0. A higher mpp value gives a lower resolution.
image_res = [0.5]

### Import .tif files

The 'nucleus.tif' and 'surface.tif' files obtained during data preparation are required by DeepCell to generate cell segmentation masks and cell outlines. To initialize the importing process, just run the following cell; the code is borrowed from an older version of DeepCell.

In [ ]:
def get_image(file_name):
    """Reads image from the file and returns it as a tensor.
    Args:
        file_name (str): path to image file
    Returns:
        numpy.array: numpy array of image data """
    ext = os.path.splitext(file_name.lower())[-1]
    if ext == '.tif' or ext == '.tiff':
        return np.float32(TiffFile(file_name).asarray())
    return np.float32(imread(file_name))



def nikon_getfiles(direc_name, channel_name):
    """Returns a sorted list of files inside 'direc_name' where 'channel_name' is in the filename.
    Args:
        direc_name (str): directory to find image files
        channel_name (str): wildcard filter for filenames
    Returns:
        list: sorted list of files inside direc_name """
    imglist = os.listdir(direc_name)
    imgfiles = [i for i in imglist if channel_name in i]
    imgfiles = sorted_nicely(imgfiles)
    return imgfiles



def get_images_from_directory(data_location, channel_names):
    """Read all images from the 'data_location' directory, where 'channel_name' is in the filename.
    Args:
        data_location (str): folder containing image files
        channel_names (str[]): list of wildcards to select filenames
    Returns:
        numpy.array: numpy array of each image in the 'data_location' directory """
    from tensorflow.python.keras import backend as K
    data_format = K.image_data_format()
    img_list_channels = []
    
    for channel in channel_names:
        img_list_channels.append(nikon_getfiles(data_location, channel))
    
    img_temp = np.asarray(get_image(os.path.join(data_location, img_list_channels[0][0])))
    n_channels = len(channel_names)
    all_images = []

    for stack_iteration in range(len(img_list_channels[0])):
        if data_format == 'channels_first':
            shape = tuple([1, n_channels] + list(img_temp.shape))
        else:
            shape = tuple([1] + list(img_temp.shape) + [n_channels])

    all_channels = np.zeros(shape, dtype=K.floatx())

    for j in range(n_channels):
        img_path = os.path.join(data_location, img_list_channels[j][stack_iteration])
        channel_img = get_image(img_path)
        if data_format == 'channels_first':
                all_channels[0, j, ...] = channel_img
        else:
                all_channels[0, ..., j] = channel_img

    all_images.append(all_channels)

    return all_images

This cell imports the nucleus.tif and surface.tif files.

In [ ]:
counter = 1
for current_folder in allFolders:
    print(counter)
    print(current_folder)
    counter = counter+1
    data_direct = path+current_folder
    names = [cellNuc,cellSur]
    
imageTemp = get_images_from_directory(data_direct, names)
image = imageTemp[0]

This cell is simply meant for a quick visualization check of the imported .tif files. Feel free to change 'channel_colors' between red / green / blue.

In [ ]:
rgb_images = create_rgb_image(image, channel_colors=['green', 'blue'])

idx = 0 # selects index for displaying
fig, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(image[idx, ..., 0])
ax[1].imshow(image[idx, ..., 1])
ax[2].imshow(rgb_images[idx, ...])
ax[0].set_title('Nuclear channel')
ax[1].set_title('Membrane channel')
ax[2].set_title('Overlay')
plt.show()

### Generate cell segmentation predictions

In the realm of biological imaging, the most common difference between datasets is the resolution of the data measured in microns per pixel. The training resolution of the model is 0.5 mpp (can also be identified using `app.model_mpp`). While 0.5 mpp may yield an acceptable cell segmentation mask, it is unsuitable for all images.

This cell performs the actual segmentations depending on the `image_res` mpp used in the "Set parameters" section.

In [ ]:
num = list(range(len(image_res)))
contain = [] 

for item in num:
    mpp=image_res[item]
    segmentation_predictions = app.predict(image, image_mpp=mpp) 
    contain.append(segmentation_predictions)

# Consider inserting 'compartment='nuclear' after 'image_mpp=mpp' to assess computer performance

This cell is simply meant for a visualization check of the segmentation by creating an overlay with our nucleus.tif and surface.tif images.

In [ ]:
outline_image = []

from deepcell.utils.plot_utils import make_outline_overlay
overlay_data = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions)
outline_image.append(overlay_data)

fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_images[idx, ...])
ax[1].imshow(overlay_data[idx, ...])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()

### Export cell segmentation mask and cell outline

The segmentation mask enables downstream analyses of cell features using our MatLab scripts. The cell outline enables quality check of the segmentation (refer to PART 2B of the protocol). If the segmentation is not ideal, it can be improved by adjusting the `image_res` mpp in the "Set parameters" section.

Feel free to edit the 'saved_name' values to your intended file name. This cell exports the segmentation mask and outline as two separate .tif files into the original file directory (where the nucleus.tif and surface.tif files are stored).

In [ ]:
output_location = data_direct

# Save: segmentation mask
for item in contain:
    item = item.astype("float32")
    saved_name_segm = 'segmentationMask.tif'

tiff.imsave(os.path.join(output_location, saved_name_segm), item[0,:,:,0])

# Save: cell outline
for item in outline_image:
    item = item.astype("float32")
    saved_name_outl = 'cellOutline.tif'

tiff.imsave(os.path.join(output_location, saved_name_outl), item[0,:,:,0])

In [ ]:
# Time when the script ends
end = time.time()
print('Script ended at time =', end)

# Time taken to run the script (in seconds)
seconds = end - start
print('Time elapsed =', seconds)